In [1]:
from finlab.crawler import (
    widget, 
    
    crawl_price, 
    crawl_monthly_report, 
    crawl_finance_statement_by_date,
    
    date_range, month_range, season_range
)

import sqlite3
import os
conn = sqlite3.connect(os.path.join('data', "data.db"))

widget(conn, 'price', crawl_price, date_range)
# widget(conn, 'monthly_revenue', crawl_monthly_report, month_range)
# widget(conn, 'finance_statement', crawl_finance_statement_by_date, season_range)

start crawl price from  2021-06-09 to 2021-06-10


  0%|          | 0/2 [00:00<?, ?it/s]

crawling 2021-06-09
success
crawling 2021-06-10
success


In [2]:
# 找尋多頭且剛突破 20 日線的股票

from finlab.data import Data
import datetime


# 前 150 大股票(0050, 0056) 號碼
stock50Pair = {
        "1101": "台泥", "1102": "亞泥", "1216": "統一", "1301": "台塑", "1303": "南亞", "1326": "台化", "1402": "遠東新", "2002": "中鋼",
        "2105": "正新", "2207": "和泰車"
        , "2301": "光寶科", "2303": "聯電", "2308": "台達電", "2317": "鴻海", "2327": "國巨", "2330": "台積電", "2357": "華碩",
        "2382": "廣達", "2395": "研華", "2408": "南亞科"
        , "2412": "中華電", "2454": "聯發科", "2474": "可成", "2633": "台灣高鐵", "2801": "彰銀", "2823": "中壽", "2880": "華南金",
        "2881": "富邦金", "2882": "國泰金", "2883": "開發金"
        , "2884": "玉山金", "2885": "元大金", "2886": "兆豐金", "2887": "台新金", "2888": "新光金", "2890": "永豐金", "2891": "中信金",
        "2892": "第一金", "2912": "統一超", "3008": "大立光"
        , "3045": "台灣大", "3711": "日月光投控", "4904": "遠傳", "4938": "和碩", "5871": "中租-KY", "5876": "上海商銀", "5880": "合庫金",
        "6505": "台塑化", "9904": "寶成", "9910": "豐泰", "2603": "長榮", "2609":"陽明", "2615":"萬海" }

stock100Pair = {
        "1210": "大成", "1227": "佳格", "1319": "東陽", "1434": "福懋", "1476": "儒鴻", "1477": "聚陽", "1504": "東元", "1536": "和大",
        "1590": "亞德客-KY", "1605": "華新"
        , "1707": "葡萄王", "1717": "長興", "1722": "台肥", "1723": "中碳", "1802": "台玻", "2027": "大成鋼", "2049": "上銀",
        "2059": "川湖", "2101": "南港", "2201": "裕隆"
        , "2204": "中華", "2227": "裕日車", "2231": "為升", "2313": "華通", "2324": "仁寶", "2337": "旺宏", "2344": "華邦電",
        "2345": "智邦", "2347": "聯強", "2352": "佳世達"
        , "2353": "宏碁", "2354": "鴻準", "2356": "英業達", "2360": "致茂", "2371": "大同", "2376": "技嘉", "2377": "微星",
        "2379": "瑞昱", "2383": "台光電", "2385": "群光"
        , "2409": "友達", "2439": "美律", "2449": "京元電子", "2451": "創見", "2492": "華新科", "2498": "宏達電", "2542": "興富發",
        "2606": "裕民", "2610": "華航", "2615": "萬海"
        , "2618": "長榮航", "2809": "京城銀", "2812": "台中銀", "2834": "臺企銀", "2845": "遠東銀", "2867": "三商壽", "2889": "國票金",
        "2903": "遠百", "2915": "潤泰全", "3005": "神基"
        , "3023": "信邦", "3034": "聯詠", "3037": "欣興", "3044": "健鼎", "3231": "緯創", "3406": "玉晶光", "3443": "創意",
        "3481": "群創", "3532": "台勝科", "3533": "嘉澤"
        , "3702": "大聯大", "3706": "神達", "4958": "臻鼎-KY", "5269": "祥碩", "5522": "遠雄", "6176": "瑞儀", "6213": "聯茂",
        "6239": "力成", "6269": "台郡", "6278": "台表科"
        , "6285": "啟碁", "6409": "旭隼", "6415": "矽力-KY", "6456": "GIS-KY", "6592": "和潤企業", "6669": "緯穎", "8046": "南電",
        "8341": "日友", "8454": "富邦媒", "8464": "億豐"
        , "9914": "美利達", "9917": "中保科", "9921": "巨大", "9933": "中鼎", "9941": "裕融", "9945": "潤泰新"
    }

# TODO 加入 0050 之類的 ETF
etfPair = { "0050": "0050", "0056":"0056","006208":"富邦台50","00881":"國泰台灣5G+","00878":"國泰永續高股息",
          "00891":"中信關鍵半導體", "00757":"統一FANG+"}
# //,"00888":"永豐台灣ESG"

allStockList = stock50Pair.copy()
allStockList.update(stock100Pair)
allStockList.update(etfPair)
# dict(list(stock50Pair.items()) + list(stock100Pair.items()))
# print(len(allStockList))

# 拿取資料
data = Data()
price70 = data.get('收盤價', 60 ) 
price60 = data.get('收盤價', 60 ) 
price30 = data.get('收盤價', 30 ) 
price20 = data.get('收盤價', 20 ) 
price10 = data.get('收盤價', 10 ) 
price5 = data.get('收盤價', 5 ) 
price1 = data.get('收盤價', 1 ) 

valume20 = data.get('成交股數', 20)
valume1 = data.get('成交股數', 1)

# 走多頭的趨勢
is1Large5 = price1[allStockList].mean() > price5[allStockList].mean()
is5Large10 = price5[allStockList].mean() > price10[allStockList].mean()
is5Large20 = price5[allStockList].mean() > price20[allStockList].mean()

is10Large20 = price10[allStockList].mean() > price20[allStockList].mean()
is20Large60 = price20[allStockList].mean() > price60[allStockList].mean()

upChoiceStocklist = is1Large5 & is5Large10 & is10Large20 & is20Large60

upStockList = []

for stockId in allStockList:
    if(upChoiceStocklist[stockId] == True):
        upStockList.append(stockId)
#         print("https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=" + stockId)


print("2 日內 20 日線大於 60 日線，找波動比較大的")
shortUpStockSet = {}
shortUpStockBefore5Days = (price30[allStockList])[-22:-2].mean() < (price70[allStockList])[-62:-2].mean()
shortUpStockNow = price20[allStockList].mean() > price60[allStockList].mean()
isRecentlyUp = shortUpStockBefore5Days & shortUpStockNow

for stockId in allStockList:
    if(isRecentlyUp[stockId]):
        print(allStockList[stockId]+"("+ str(price1[stockId].mean())+"): "+"https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=" + stockId)
        

print("2 日內 5 日線大於 20 日線，找波動比較大的")
shortUpStockSet = {}
shortUpStockBefore5Days = (price10[allStockList])[-7:-2].mean() < (price30[allStockList])[-22:-2].mean()
shortUpStockNow = price5[allStockList].mean() > price20[allStockList].mean()
isRecentlyUp = shortUpStockBefore5Days & shortUpStockNow

for stockId in allStockList:
    if(isRecentlyUp[stockId]):
        print(allStockList[stockId]+"("+ str(price1[stockId].mean())+"): "+"https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=" + stockId)
          
        
        

# for stockId in isRecentlyUp:
#     if(upChoiceStocklist[stockId] == True):
#         print(allStockList[stockId]+"("+ str(price1[stockId].mean())+"): "+"https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=" + stockId)
        
# print((price10[allStockList])[-10:-5].)

# for stockId in upStockList:
#     if(is5Large20[stockId]):
#         print(allStockList[stockId]+"("+ str(price1[stockId].mean())+"): "+"https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=" + stockId)

# print("# 20 日線大於 60 日線")
# for stockId in upStockList:
#     if(is20Large60[stockId]):
#         print(allStockList[stockId]+"("+ str(price1[stockId].mean())+"): "+"https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=" + stockId)
  
        
        
# # 今日剛突破 20 日線 & 前一天前還沒突破 & 今日成交量比 20 日平均高
# less20Before3Day = (price5[allStockList])[-2:-1].mean() < price20[allStockList].mean()
# is1Large20 = price1[allStockList].mean() > price20[allStockList].mean() 
# is1VolumeLarge20 = valume1[allStockList].mean() > valume20[allStockList].mean() 

# breakthrough20StockList = is1Large20 & less20Before3Day 


# print(datetime.date.today())
# print("# 20 日線的股票")
# for stockId in allStockList:
#     if(breakthrough20StockList[stockId]):
#         print(allStockList[stockId]+"("+ str(price1[stockId].mean())+"): "+"https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=" + stockId)


2 日內 20 日線大於 60 日線，找波動比較大的
遠東新(31.95): https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=1402
遠傳(65.6): https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=4904
致茂(201.0): https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=2360
技嘉(114.0): https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=2376
華航(18.85): https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=2610
長榮航(20.7): https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=2618
南電(348.5): https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=8046
2 日內 5 日線大於 20 日線，找波動比較大的


In [3]:
# KD 20 選股法
import talib
import pandas as pd

# 從 close, high, low 取從最新的一筆資料往前算，2000筆資料
close = data.get('收盤價', 300)
high = data.get('最高價', 300)
low = data.get('最低價', 300)

# 利用 talib 來計算 0050 這檔股票的指標
print(datetime.date.today())
print("# KD 20 選股法")
for stockId in allStockList:
    kd = talib.STOCH(high[stockId].ffill().values, 
                 low[stockId].ffill().values, 
                 close[stockId].ffill().values,
                 fastk_period=9, slowk_period=3
                 , slowd_period=3, slowk_matype=1, slowd_matype=1)
    # 將 kd 值 分別轉換成 k、d 兩條 series
    k = pd.Series(kd[0], index=close[stockId].index)
    d = pd.Series(kd[1], index=close[stockId].index)

    # & 代表 「且」 的意思，這邊的條件就是
    # k.shift(1) 可以想成「昨天」的意思，同理 k.shift(2) 就是前天
    # 當（今天的 k > d ）且 （昨天的 k < d） 且 （今天的 k < 30） 時，買入

    buy = (k > d) & (k.shift(1) < d.shift(1)) & (k < 30)
  
# 
    if buy[str(datetime.date.today())]:
        print(allStockList[stockId]+"("+ str(price1[stockId].mean())+"): "+"https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=" + stockId)
 
print("End")


2021-06-10
# KD 20 選股法
End


In [4]:
# 停利 停損計算

# {股票代號, 買時候的股價}

# 20 日線 波段股票
bandStockPricePair = {"2027" : 48}

# KD 20 波段股票
kdStockPricePair = {"2542" : 42.46 }

# 投機股票
opportunisticStockPricePair = {"2610" : 20.8 , "2603": 110 }

print(datetime.date.today())
# 停損
print("20 日線停損(跌破 20 日線＆低於買進價 10%)")
for stockId in bandStockPricePair:
    below20Price = price1[stockId].mean() < price20[stockId].mean()
    beyond10Percent = price1[stockId].mean() < (bandStockPricePair[stockId] * 0.9)
    if(below20Price & beyond10Percent):
        print(stockId+"("+ str(price1[stockId].mean())+"): "+"https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=" + stockId)
print("============================")    


print("")
print("kd 停損(低於買進價 10%)")
for stockId in kdStockPricePair:
    beyond10Percent = price1[stockId].mean() < kdStockPricePair[stockId] * 0.90
    if(beyond10Percent):
        print(stockId+"("+ str(price1[stockId].mean())+"): "+"https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=" + stockId)
print("============================")    



print("")
print("投機停損(低於買進價 20%)")
for stockId in opportunisticStockPricePair:
    beyond10Percent = price1[stockId].mean() < opportunisticStockPricePair[stockId] * 0.8
    if(beyond10Percent):
        print(stockId+"("+ str(price1[stockId].mean())+"): "+"https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=" + stockId)
print("============================")    



# 停利
# below20Price = price1[bandStockPricePair] < price20[bandStockPricePair]
# print("20 日線停利")
# for stockId in below20Price:
#         print(allStockList[stockId]+"("+ str(price1[stockId].mean())+"): "+"https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=" + stockId)

# print("kd 停利")

print("")
print("20 日線停利(移動式停利，高點跌下來 10%")
for stockId in bandStockPricePair:
    belowHighest10PercentPrice = price1[stockId].mean() < price20[stockId].max() * 0.9
    if belowHighest10PercentPrice:
        print(stockId+"("+ str(price1[stockId].mean())+"): "+"https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=" + stockId)
print("============================")    

print("")
print("kd 停利(KD 死亡交叉，K 大於 80)")
for stockId in kdStockPricePair:
    kd = talib.STOCH(high[stockId].ffill().values, 
                 low[stockId].ffill().values, 
                 close[stockId].ffill().values,
                 fastk_period=9, slowk_period=3
                 , slowd_period=3, slowk_matype=1, slowd_matype=1)
    # 將 kd 值 分別轉換成 k、d 兩條 series
    k = pd.Series(kd[0], index=close[stockId].index)
    d = pd.Series(kd[1], index=close[stockId].index)

    # & 代表 「且」 的意思，這邊的條件就是
    # k.shift(1) 可以想成「昨天」的意思，同理 k.shift(2) 就是前天
    # 當（今天的 k < d ）且 （昨天的 k > d） 且 （今天的 k < 30） 時，賣出入

    buy = (k < d) & (k.shift(1) > d.shift(1)) & (k >= 80)
  
    if buy[str(datetime.date.today())]:
        print(stockId+"("+ str(price1[stockId].mean())+"): "+"https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=" + stockId)
print("============================")    


print("")
print("投機停利(移動式停利，高點跌下來 10%)")
for stockId in opportunisticStockPricePair:
    belowHighest10PercentPrice = price1[stockId].mean() < price20[stockId].max() * 0.9
    if belowHighest10PercentPrice:
        print(stockId+"("+ str(price1[stockId].mean())+"): "+"https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID=" + stockId)
print("============================")    


2021-06-10
20 日線停損(跌破 20 日線＆低於買進價 10%)

kd 停損(低於買進價 15%)

投機停損(低於買進價 15%)

20 日線停利(移動式停利，高點跌下來 10%

kd 停利(KD 死亡交叉，K 大於 80)

投機停利(移動式停利，高點跌下來 10%)


In [5]:
# 將結果送到 IM